Crie um sistema capaz de resolver o problema das N-Rainhas através de busca
de melhoria iterativa (hill climbing ou têmpera simulada) para N com os seguintes
valores 10, 20 e 25. Tabele os tempos de processamento para obter uma solução.

In [107]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
import re
import random

In [748]:
# grid = np.zeros((10, 10), dtype=int)
# print(grid)

In [750]:
# Calcula o score do estado atual com base no número de rainhas que se atacam
# Score = Max - pares de raínhas que se atacam
def score_state(state):
    L = len(state)
    max_atacks = L*(L-1)/2
    state = np.insert(state, 0, 0, axis=0)
    score = 0
    for i in range(1, L+1):
        if i <= L:
            for j in range(i+1, L+1):
                if state[j] == state[i]:
                    score += 1
                elif abs(i - state[i]) == abs(j - state[j]):
                    score += 1
                elif state[i] + i == state[j] + j:
                    score += 1
    
    return max_atacks - score

# Calcula a probabilidade de se escolher dados cromossomo com base no seu score
def fitness_function(population):
    total_score = 0
    L = len(population)
    for person in population:
        total_score += score_state(person)
    
    if total_score == 0:
        return np.random.dirichlet(np.ones(L),size=1)[0]
    
    result = np.array([])
    for i in range(L):
        result = np.append(result, score_state(population[i])/total_score)
    
    return result

# Retorna a subpopulação de cromossomos com base na sua probabilidade de ser escolhido
def selection(population, p):
    # Seleção usando o método da roleta. De toda população selecionar uma fração p
    L = len(population)
    selected = np.random.choice(np.arange(0, L, 1), int(len(population)*p), p=fitness_function(population), replace=False)
    
    result = [population[i] for i in selected]
            
    return result

# Retorna uma nova população de indivíduos após serem cruzados
def cross_over(population):
    selected = np.zeros(len(population)) #1: population[i] já selecionado 0: ainda não selecionado
    new_population = []
    for i in range(len(selected) - 1):
        if selected[i] != 1:
            #Seleciona esse indivíduo
            selected[i] = 1

            #Seleciona outro, à frente no vetor, aleatoriamente
            j = np.random.choice(len(selected))
            while ~(selected[j] == 0 or j <= i):
                j = np.random.choice(len(selected))

            selected[j] = 1
            
            p1 = population[i]
            p2 = population[j]
    
            #Seleciona ao acaso uma posição para o cross
            x = np.random.choice(len(population[0])) + 1

            tmp = p2[:x].copy()
            p2[:x], p1[:x]  = p1[:x], tmp

            new_population.append(p1)
            new_population.append(p2)
        
    
    return new_population

# Adiciona mutação em um gene na fração p de indivíduos da população
def mutation(population, p):
    L = int(len(population)*p)
    #Uma fração de p indivíduos irá sofrer uma mutação em um gene
    for i in range(L):
        #Seleciona ao acaso um indivíduo pra sofrer mutação
        idx = np.random.randint(population.shape[0], size=1)
        cromo = population[idx, :][0]
        
        #Seleciona ao acaso uma posição desse indivíduo
        posic = np.random.choice(len(cromo), 1)
        
        #Seleciona ao acaso um novo gene
        cromo[posic] = np.random.choice(len(cromo), 1)
        
        #Substitui na população
        population[idx] = cromo
        
        
    return population
    

In [573]:
population = np.array([[1, 2, 3, 4], [1, 3, 2, 2], [1, 3, 2, 4], [1, 1, 1, 1]])